<a href="https://colab.research.google.com/github/sanghyun-ai/ktcloud_genai/blob/main/%EC%8B%A4%EC%8A%B5%EC%BD%94%EB%93%9C/207_LLM_%EC%83%9D%EC%84%B1%EB%AA%A8%EB%8D%B8_%ED%8C%8C%EC%9D%B8%ED%8A%9C%EB%8B%9D%ED%95%98%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **생성 모델 미세 튜닝하기**



---


- 💡 **NOTE**
    - 이 노트북의 코드를 실행하려면 GPU를 사용하는 것이 좋습니다. 구글 코랩에서는 **런타임 > 런타임 유형 변경 > 하드웨어 가속기 > T4 GPU**를 선택하세요.


---



In [1]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

*trl 버전 0.17에서 SFTTrainer를 실행할 때 KeyError: 'completion'가 발생하므로 trl 버전을 0.16.1로 고정한다.*

In [2]:
%%capture
!pip install datasets bitsandbytes trl==0.16.1

- **bitsandbytes** :
    - GPU 메모리를 절약하기 위한 8비트/4비트 양자화 및 최적화 라이브러리
    - 용도 : 모델 경량화, QLoRA 구현
    - 하드웨어 사양 참고 : https://github.com/bitsandbytes-foundation/bitsandbytes

- **peft** :
    - 거대 AI 모델의 모든 파라미터를 건드리지 않고 LoRA처럼 아주 작은 일부 파라미터만 추가하거나 수정하여, 적은 메모리와 컴퓨팅 자원으로도 빠르고 효율적인 미세조정(fine-tuning)을 가능하게 해주는 허깅페이스의 라이브러리
    - LoRA  설정 구성 --> 미세 튜닝 과정의 하이퍼파라미터에 해당
    - https://github.com/huggingface/peft
    
- **trl**
    - Transformer 모델을 강화학습(RLHF)으로 fine-tuning하기 위한 Hugging Face 라이브러리
    - 용도 : PPO, DPO, SFT 등



---



## **LLM 훈련의 세 단계**

- **고품질의 데이터를 만드는 과정**
    - **사전 훈련**(Pre-training)
    - **지도학습 미세 튜닝**(SFT: Supervised Fine-Tuning)
        - 전체 미세 튜닝 (full fine-tuning)
        - 파라미터 효율적인 미세 튜닝 (**PEFT**: Parameter-efficient fine-tuning)
    - **선호도 튜닝**(PT: Preference Tuning / RLHF: Reinforcement Learning from Human Feedback)



---



### **1단계 사전 훈련** :
- **<mark>하나 이상의 대규모 텍스트 데이터셋에서 사전 훈련하는 과정**.
- 훈련하는 동안 텍스트에 있는 다음 토큰을 예측하면서 **<mark>언어적, 의미론적 표현을 정확하게 학습**한다.
- **언어 모델링** or **자기 지도 학습 방법**
- 이를 통해 **베이스 모델**을 만든다.
- 사전 훈련된 모델(pretrained model) or 파운데이션 모델(foundation model)이라고 부른다.
- 베이스 모델은 최종 사용자가 활용하기는 어렵다.

- **언어 모델링 과정 예:** *다음 토큰을 예측*
<img src="https://drive.google.com/uc?export=view&id=161RgBJdvmvbRnM7QXixvubvLCABOeH_5" width="80%" title="이미지: 핸즈온 LLM">


<img src="https://drive.google.com/uc?export=view&id=19fP1fbAKgiPOFp850n4-87EzakF7UvDs" width="90%" title="이미지: 핸즈온 LLM">



---



### **2단계 (미세 튜닝1) 지도 학습 미세 튜닝**

- **<mark>베이스 모델을 지시에 따르게 적응시키는 과정**
- 미세 튜닝 과정을 통해 베이스 모델의 파라미터가 <mark>**지시 수행과 같은 타깃 작업에 더 잘 맞도록 업데이트**</mark> 된다.
- 사전 훈련된 모델처럼 다음 토큰을 사용해 훈련되지만 다음 토큰만 예측하는 것이 아니라 사용자의 입력(레이블)을 기반으로 훈련을 수행한다.
- SFT는 분류와 같은 다른 작업에도 사용될 수 있지만, 베이스모델을 지시 모델(채팅 모델)로 만드는데 많이 사용됨


- **지도학습 미세 튜닝 과정 예:**
<img src="https://drive.google.com/uc?export=view&id=1gflF27fDTxOLZBoDXHnmH2dCz3LQ5g7s" width="80%" title="이미지: 핸즈온 LLM">

#### **전체 미세 튜닝**
- LLM은 사전 훈련과 비슷하게  타깃 작업에 맞도록 모델의 모든 파라미터를 업데이트함
- 레이블이 있는 작은 데이터를 사용
- 도메인에 특화된 표현을 학습하기 위한 효과적인 기술
- **LLM을 지시에 따르게 하려면 <mark>질문-응답(질문/지시에 상응하는 답변)</mark> 데이터가 필요**
- **교재 그림 참고**
- 전체 파라미터 업데이트는 성능 향상에 좋을 수 있지만 비용이 많이 들고, 속도가 느리고, 많은 저장공간이 필요하다.

        

<img src="https://drive.google.com/uc?export=view&id=1Zre_vNVUfhxR2e7rTCyD_tZBgg3pR_iU" width="70%" title="이미지: 핸즈온 LLM">

#### **PEFT**
- 파라미터 효율적인 미세 튜닝(PEFT: Parameter-efficient fine-tuning
- 전체 미세 튜닝의 단점을 해결하기 위해 <mark>**`적은 파라미터로 효율적인 학습(미세조정)'**</mark>에 초점을 맞
- 매우 높은 계산 효율로 사전 훈련된 모델을 미세 튜닝하는 방법
    - 어댑터(adapter)
    - LoRA(Low-Rank Adaptation)
    - 모델 압축을 통한 훈련(Model Compression)


##### **Adapter**

- 기존 모델의 레이어 사이에 **아주 작은 신경망(어댑터)을 추가**한 뒤, 원본은 그대로 두고 **어댑터만 집중적으로 학습**시키는 기법
- 일종의 플러그인 기능(?)

| 구분 | 설명 |
|---|---|
|**핵심 아이디어** | 기존 LLM 레이어 사이에 '병목(bottleneck)' 구조의 작은 어댑터(신경망) 모듈을 삽입하고,</br><br>이 모듈만 훈련시킵니다. (LLM 본체는 동결) |
|**주요 목적** | 효율적인 미세조정 (적은 자원으로 훈련)|
|적용 위치 |트랜스포머 블록의 **레이어와 레이어 사이** |
|**장점** | 개념이 직관적이고 구현이 쉬움</br><br>하나의 원본 모델로 여러 태스크의 어댑터를 관리하기 용이함 |
|**단점** | 추론 시 어댑터 레이어를 거치며 미세한 속도 저하가 발생할 수 있음|



- **교재 그림 참고**
- **어댑터 허브** : https://adapterhub.ml/
    - 미리 학습된 어댑터(Adapter)들을 위한 중앙 저장소이자 공유 플랫폼



##### **LoRA**

- 어댑터의 대안으로 LoRA(Low-Rank Adaptation)가 소개됨 https://arxiv.org/abs/2106.09685
- 모델 가중치의 변화량을 '랭크(rank)가 낮은' **작은 행렬 두 개의 곱으로 표현**(분해)하여, 이 **작은 행렬들만 학습**시키는 기법 --> **적은 수의 파라미터만 학습(업데이트)**
- 주로 셀프 어텐션의 쿼리(Query)와 밸류(Value) 가중치 행렬에 적용했을 때 가장 효율이 좋다고 알려져 있음
- 원본 가중치($W_0$)는 동결시키고, 오직 작은 행렬 A와 B만 학습시킴
    - $W_{tuned}$ = $W_0 + ΔW$ = $W_0 + B⋅A$


| 구분 | 설명 |
|---|---|
|**핵심 아이디어** | 거대한 가중치 행렬의 업데이트 정보(ΔW, 가중치의 변화량)를 두 개의 작은 저차원(low-rank) 행렬(B⋅A)로 근사하여 학습 파라미터를 극적으로 줄입니다. |
|**주요 목적** | 효율적인 미세조정 (훈련 효율성 극대화)|
|적용 위치 |어텐션 등 **특정 레이어의 가중치 행렬 자체** |
|**장점** | **가장 널리 쓰이는 PEFT 기법** </br><br>어댑터보다 학습 파라미터가 더 적음</br><br>추론 시 추가적인 계산 지연이 거의 없음 |
|**단점** | 개념이 어댑터보다 상대적으로 복잡함|

- **LoRA 훈련 과정**
<img src="https://drive.google.com/uc?export=view&id=1v_IAj0PrBwLiH0itAFWvfaIeEPM108ej" width="80%" title="이미지: 핸즈온 LLM">


- 💡**LoRA에 대한 검증**
    - Intrinsic Dimensionality Explains the Effectiveness of Language Model Fine-Tuning : https://arxiv.org/abs/2012.13255
    - GPT-3과 같은 1,750억 파라미터의 모델은 96개의 트램스포머 블록 안에 각각 12,288 x 12,288 크기의 가중치 행렬을 가짐
    - 이 행렬의 파라미터 개수가 1억 5천만개임
    - 이 행렬을 랭크 8인 행렬로 성공적으로 분해할 수 있다면 12,288x8 크기 행렬만 필요하므로 블록마다 98K 파라미터만 있으면 된다.
    - LoRA 논문(https://arxiv.org/abs/2106.09685 )에서 설명한 것 처럼 이를 통해 속도, 저장 공간, 컴퓨팅을 크게 절약하게 됨  

- 💡**미세 튜닝에 대한 추가적인 정보**
    - https://magazine.sebastianraschka.com/p/practical-tips-for-finetuning-llms


#### **모뎁 압축을 통한 훈련**

- **이미 학습이 완료된 모델의 크기를 줄이거나**(경량화) 연산 속도를 높여, **<mark>실제 서비스 환경에서의 배포를 용이하게**</mark> 만드는 기법들의 총칭
- 모델 압축(Model Compression) 은 '학습이 완료된 모델을 추론(inference) 환경에 배포하기 위해 효율적으로 만드는 것'에 더 중점



- **핵심 아이디어**:
    - **<mark>가지치기**(Pruning)</mark>: 불필요한 가중치를 0으로 만들어 제거
    - **<mark>양자화**(Quantization)</mark>: 32비트 실수를 8비트 정수 등으로 변환
    - **<mark>지식 증류**(Knowledge Distillation)</mark>: 큰 모델(교사)의 지식을 작은 모델(학생)에게 전달
- **주요 목적** :  
    - **효율적인 추론** (빠른 응답 속도, 작은 저장 공간)
- 장점 :
    - 모델 파일 크기가 획기적으로 줄어듦
    - 추론 속도가 빨라져 운영 비용 절감
    - 저사양 기기(모바일 등)에 배포 가능
- 단점 :
    - 압축 과정에서 모델 성능이 일부 손실될 위험이 있음
    - 압축 기법 자체가 복잡하고 까다로움


##### **QLoRA**

- **양자화(quantization)는 원본 가중치 값을 정확히 나타내면서 비트 수를 줄이는 것이 목적**
    - 높은 정밀도의 값을 낮은 정밀도의 값으로 매핑할 때, 높은 정말도 값 여러 개가 낮은 정밀도 값 하나로 동일하게 표현될 수 있다.
- **LoRA에 4비트 양자화를 결합하여 메모리를 극적으로 줄이면서도 성능은 유지하는 초효율 fine-tuning 방법**
    - <mark>**QLoRA = LoRA + 양자화(Quantization)**
        - LoRA: 저차원 행렬로 효율적 학습
        - 4-bit Quantization: 모델 가중치를 4비트로 압축
    - 결과: 일반 소비자용 GPU(24GB)로도 65B(650억 개의 파라미터) 모델 fine-tuning 가능
        - 일반 fine-tuning: 65B 모델에 780GB 메모리 필요
        - LoRA: 약 120GB 메모리 필요
        - QLoRA: 약 48GB 메모리로 가능! (RTX 3090 2개면 충분)
- QLoRA 저자는 원본 가중치와 크게 차이 나지 않도록 높은 비트의 값을 낮은 정밀도로 바꾸거나 그 반대로 바꾸는 방법을 찾음
    - 블록 단위 양자화를 사용하면 LLM의 성능을 약간만 손해 보면서 높은 정밀도의 값을 낮을 정밀도의 값으로 정확하게 표현할 수 있음.
    - 결과적으로 16비트 부동소수점 표현을 4비트만 사용한 정규화된 부동소수점 표현으로 바꿀 수 있음.
    - 4비트 표현은 LLM 훈련 중에 메모리 요구량을 크게 줄여줌
    - 일반적으로 LLM 양자화는 추론에도 도움이 됨 --> 양자화된 LLM의 크기가 작아 VRAM이 덜 필요하기 때문
- [참고] 최적화 방법 : https://newsletter.maartengrootendorst.com/p/a-visual-guide-to-quantization
    - 이중 양자화(double quantization)
    - 페이지 기반 옵티마이저(paged optimizer)
    - 등이 있음
- **교재 그림 참고**

- **QLoRA 분포를 고려한 블록 사용**
<img src="https://drive.google.com/uc?export=view&id=1o81i7NFW-LKw0aKh4EoGXq6tZbXcS5d5" width="80%" title="이미지: 핸즈온 LLM">



---



### **3단계 (미세 튜닝2) 선호도 튜닝**

- **<mark>모델의 품질을 더 향상시키는 과정** -->AI안정성과 사람의 선호도에 맞는 행동을 하도록 만드는 과정
- 선호도 튜닝은 미세 튜닝의 한 형태이며 (이름에서 알 수 있듯이) 우리가 전달하는 데이터에 의해 정의되는 선호도에 모델의 출력을 맞춘다(alignment).
- 지도 학습 미세 튜닝에서처럼 원본 모델의 성능을 향상시킬 수 있지만 훈련 과정에서 출력에 대한 선호도를 추출하는 부가적인 이득이 있음

- **LLM 미세튜닝 과정**
<img src="https://drive.google.com/uc?export=view&id=1yG5BsM5w49bbzNAIpzbfkgfTWwf_D_SP" width="80%" title="이미지: 핸즈온 LLM">



---



## **지도 학습 미세 튜닝 : QLoRA를 사용한 지시 기반 튜닝**


- **사전 학습 모델**
    - **TinyLlama/TinyLlama-1.1B-intermediate-step-1431k-3T**    
    - Llama의 소규모 버전
    - https://huggingface.co/TinyLlama/TinyLlama-1.1B-intermediate-step-1431k-3T
    - **TinyLlama를 채팅 템플릿 형태의 지시에 따르도록 미세 튜닝한다**
- **데이터**
    - **tokenizer** : TinyLlama/TinyLlama-1.1B-Chat-v1.0
    - **HuggingFaceH4/ultrachat_200k**
    - UltraChat 데이터셋, 사용자와 LLM의 대화 약 20만건 (우린 일부분만 사용)
    - https://huggingface.co/datasets/HuggingFaceH4/ultrachat_200k
- 지시 테이터 템플릿
    - format_prompt() 함수 만들어서 사용
<img src="https://drive.google.com/uc?export=view&id=1oaqB0JcUpOE3U7Tpy1ruG32Q1eUpPdJS" width="60%" title="이미지: 핸즈온 LLM">

### **데이터 전처리**

In [3]:
from transformers import AutoTokenizer
from datasets import load_dataset

# 채팅 템플릿을 사용하기 위해 토크나이저를 로드합니다.
template_tokenizer = AutoTokenizer.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")


def format_prompt(example):
    """TinyLlama의 <|user|> 템플릿으로 프롬프트를 포맷팅합니다"""

    # 채팅 템플릿 구성
    chat = example["messages"]
    prompt = template_tokenizer.apply_chat_template(chat, tokenize=False)

    return {"text": prompt}


# 데이터를 로드하고 TinyLlama 템플릿을 적용합니다.
dataset = (
    load_dataset("HuggingFaceH4/ultrachat_200k",  split="test_sft")
      .shuffle(seed=42)
      .select(range(3_000)) # 3000건만 가져옴
)
dataset = dataset.map(format_prompt).remove_columns(['messages'])

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

Generating train_sft split:   0%|          | 0/207865 [00:00<?, ? examples/s]

Generating test_sft split:   0%|          | 0/23110 [00:00<?, ? examples/s]

Generating train_gen split:   0%|          | 0/256032 [00:00<?, ? examples/s]

Generating test_gen split:   0%|          | 0/28304 [00:00<?, ? examples/s]

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

In [4]:
dataset

Dataset({
    features: ['prompt', 'prompt_id', 'text'],
    num_rows: 3000
})

In [5]:
tmp = load_dataset("HuggingFaceH4/ultrachat_200k")
tmp

DatasetDict({
    train_sft: Dataset({
        features: ['prompt', 'prompt_id', 'messages'],
        num_rows: 207865
    })
    test_sft: Dataset({
        features: ['prompt', 'prompt_id', 'messages'],
        num_rows: 23110
    })
    train_gen: Dataset({
        features: ['prompt', 'prompt_id', 'messages'],
        num_rows: 256032
    })
    test_gen: Dataset({
        features: ['prompt', 'prompt_id', 'messages'],
        num_rows: 28304
    })
})

In [6]:
# 프롬프트 예시
print(dataset["text"][2576])

<|user|>
Given the text: Knock, knock. Who’s there? Hike.
Can you continue the joke based on the given text material "Knock, knock. Who’s there? Hike"?</s>
<|assistant|>
Sure! Knock, knock. Who's there? Hike. Hike who? Hike up your pants, it's cold outside!</s>
<|user|>
Can you tell me another knock-knock joke based on the same text material "Knock, knock. Who's there? Hike"?</s>
<|assistant|>
Of course! Knock, knock. Who's there? Hike. Hike who? Hike your way over here and let's go for a walk!</s>



### **모델 양자화**

- **bitsandbytes** 패키지를 사용해 사전 훈련 모델을 4비트 표현으로 압축함
    - 하드웨어 사양 참고: https://github.com/bitsandbytes-foundation/bitsandbytes
    
- **BitsAndBytesConfig** :  4-비트 양자화 설정
    - 예: 4GB VRAM --> 1GB VRAM

In [7]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

model_name = "TinyLlama/TinyLlama-1.1B-intermediate-step-1431k-3T"

# 4-비트 양자화 설정 - QLoRA의 Q 단계
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,  # 4-비트 정밀도 모델 로드
    bnb_4bit_quant_type="nf4",  # 양자화 종류
    bnb_4bit_compute_dtype="float16",  # 계산 dtype
    bnb_4bit_use_double_quant=True,  # 이중 양자화 적용
)

# 모델을 로드하고 GPU에서 훈련합니다.
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",

    # 일반적인 SFT에서는 다음을 삭제하세요.
    quantization_config=bnb_config, # 양자화할때만 이거 사용!
)
model.config.use_cache = False

# LLaMA 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = "<PAD>"
tokenizer.padding_side = "left"

config.json:   0%|          | 0.00/560 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.40G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/129 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

### **설정**

#### **LoRA 설정**
- **peft** :
    - 거대 AI 모델의 모든 파라미터를 건드리지 않고 LoRA처럼 아주 작은 일부 파라미터만 추가하거나 수정하여, 적은 메모리와 컴퓨팅 자원으로도 빠르고 효율적인 미세조정(fine-tuning)을 가능하게 해주는 허깅페이스의 라이브러리
    - LoRA  설정 구성 --> 미세 튜닝 과정의 하이퍼파라미터에 해당
    - https://github.com/huggingface/peft
    

- [참고] 미세 튜닝에 대한 추가적인 정보
    - https://magazine.sebastianraschka.com/p/practical-tips-for-finetuning-llms

 - target_modules=  # 대상 층 (adaptror를 지정할 층)
    - LoRA(저랭크 어댑터)를 삽입할 “가중치 행렬(Linear layer)”의 이름 목록.

    즉, 아래에 적은 이름을 가진 선형층들에만 LoRA의 저랭크 업데이트를 붙여서, 그 부분만 학습 가능(나머지는 동결)하게 만듭니다. 리스트에 무엇을 넣느냐가 학습 품질·속도·VRAM을 좌우합니다.
     - ['k_proj', 'gate_proj', 'v_proj', 'up_proj', 'q_proj', 'o_proj', 'down_proj']

    - **트랜스포머에서 의미 있는 표현 변화가 주로 일어나는 곳**이
        - ① 어텐션의 Q/K/V/O 투영(query/key/value/output projections)과
        - ② FFN(MLP)의 업/다운/게이트 투영 이므로, 이들에 LoRA를 거는 것이 관행입니다.

In [8]:
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model

# LoRA 설정
peft_config = LoraConfig(
    lora_alpha=128,     # LoRA 스케일링(원본 가중치에 추가되는 변화량을 제어, 보통 r 크기의 2배)
    lora_dropout=0.1,   # LoRA 층의 드롭아웃 (=출력층)
    r=64,               # 랭크(압축된 행렬의 랭크 4~64)
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=     # 대상 층 (adaptror를 지정할 층)
     ['k_proj', 'gate_proj', 'v_proj', 'up_proj', 'q_proj', 'o_proj', 'down_proj']
)

# 훈련을 위한 모델 준비
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)

#### **훈련 설정**
- 훈련 매개변수 설정
- **trl**
    - Transformer 모델을 강화학습(RLHF)으로 fine-tuning하기 위한 Hugging Face 라이브러리

In [9]:
from trl import SFTConfig

output_dir = "./results"

# 훈련 매개변수
training_arguments = SFTConfig(
    output_dir=output_dir,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    optim="paged_adamw_32bit",      # QLoRA 논문에서 사용된 페이지 기반 옵티마이저 지정
    learning_rate=2e-4,             # 가중치 업데이트 스텝 크기, 학습률, 0.0002
    lr_scheduler_type="cosine",     # 학습률 조정 스케쥴러, cosine(동적으로 학습률 조정함)
    num_train_epochs=1,             # 1번 학습
    logging_steps=10,               # 10 스텝마다 손실 출력
    fp16=True,
    gradient_checkpointing=True,
    dataset_text_field="text",
    max_length=512
)

### **훈련**

In [10]:
# T4에서 오래 걸림
from trl import SFTTrainer

# 지도 미세 튜닝 매개변수 지정
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    processing_class=tokenizer,
    args=training_arguments,

    # 일반적인 SFT에서는 다음을 삭제하세요.
    peft_config=peft_config,
)

# 모델 훈련
trainer.train()

# QLoRA 가중치 저장
trainer.model.save_pretrained("TinyLlama-1.1B-qlora")

Converting train dataset to ChatML:   0%|          | 0/3000 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/3000 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/3000 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/3000 [00:00<?, ? examples/s]

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 0}.
/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: aivhyun (aivhyun-none) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
10,1.578300
20,1.446500
30,1.428200
40,1.465700
50,1.455200
60,1.369000
70,1.479700
80,1.433200
90,1.415500
100,1.388300


### **가중치 병합(어댑터 병합)**

- QLoRA 가중치를 훈련후 사용하려면 원본 가중치와 병합해야 한다.
- 가중치를 병합하기 위해 양자화된 4bit가 아니라 16bit로 모델을 다시 로드한다.


In [11]:
from peft import AutoPeftModelForCausalLM

model = AutoPeftModelForCausalLM.from_pretrained(
    "/content/TinyLlama-1.1B-qlora",    # 로컬에 있는 모델 불러오기
    low_cpu_mem_usage=True,
    device_map="auto",
)

# LoRA와 베이스 모델을 병합합니다.
merged_model = model.merge_and_unload()

- **만약 오류 발생한다면**
    - adapter_config.json 파일 안의 base_model_name_or_path 값이 None이거나 누락되어서
- 아래 코드 실행하여 오류 수정하고 위 가중치 병합을 다시 실행한다.

In [12]:
# 코랩에서 바로 실행하세요!
import json
import os

config_path = "/content/TinyLlama-1.1B-qlora/adapter_config.json"

print("=" * 60)
print("adapter_config.json 파일 확인")
print("=" * 60)

# 파일 존재 확인
if os.path.exists(config_path):
    print("✅ 파일 존재함\n")

    # 내용 읽기
    with open(config_path, 'r') as f:
        config = json.load(f)

    print("현재 설정 내용:")
    print(json.dumps(config, indent=2, ensure_ascii=False))

    # 핵심 문제 확인
    print("\n" + "=" * 60)
    print("문제 진단:")
    print("=" * 60)

    if "base_model_name_or_path" in config:
        if config["base_model_name_or_path"] is None:
            print("❌ base_model_name_or_path = None")
            print("   → 이것이 오류의 원인입니다!")
        else:
            print(f"✅ base_model_name_or_path = {config['base_model_name_or_path']}")
    else:
        print("❌ base_model_name_or_path 키가 아예 없습니다!")
        print("   → 이것이 오류의 원인입니다!")
else:
    print(f"❌ 파일이 존재하지 않습니다: {config_path}")

adapter_config.json 파일 확인
✅ 파일 존재함

현재 설정 내용:
{
  "alpha_pattern": {},
  "auto_mapping": null,
  "base_model_name_or_path": "TinyLlama/TinyLlama-1.1B-intermediate-step-1431k-3T",
  "bias": "none",
  "corda_config": null,
  "eva_config": null,
  "exclude_modules": null,
  "fan_in_fan_out": false,
  "inference_mode": true,
  "init_lora_weights": true,
  "layer_replication": null,
  "layers_pattern": null,
  "layers_to_transform": null,
  "loftq_config": {},
  "lora_alpha": 128,
  "lora_bias": false,
  "lora_dropout": 0.1,
  "megatron_config": null,
  "megatron_core": "megatron.core",
  "modules_to_save": null,
  "peft_type": "LORA",
  "qalora_group_size": 16,
  "r": 64,
  "rank_pattern": {},
  "revision": null,
  "target_modules": [
    "o_proj",
    "up_proj",
    "q_proj",
    "v_proj",
    "gate_proj",
    "down_proj",
    "k_proj"
  ],
  "target_parameters": null,
  "task_type": "CAUSAL_LM",
  "trainable_token_indices": null,
  "use_dora": false,
  "use_qalora": false,
  "use_rslora"

In [13]:
import json

# adapter_config.json 파일 수정
config_path = "/content/TinyLlama-1.1B-qlora/adapter_config.json"

print("=" * 60)
print("adapter_config.json 수정 중...")
print("=" * 60)

# 파일 읽기
with open(config_path, 'r') as f:
    config = json.load(f)

# 문제 확인
print(f"\n수정 전: base_model_name_or_path = {config.get('base_model_name_or_path')}")

# 베이스 모델 정보 추가/수정
# ⚠️ 실제 사용한 베이스 모델 이름으로 변경하세요!
config["base_model_name_or_path"] = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

print(f"수정 후: base_model_name_or_path = {config['base_model_name_or_path']}")

# 파일 저장
with open(config_path, 'w') as f:
    json.dump(config, f, indent=2, ensure_ascii=False)

print("\n✅ 수정 완료!")

# 검증
print("\n" + "=" * 60)
print("수정 결과 확인:")
print("=" * 60)

with open(config_path, 'r') as f:
    updated_config = json.load(f)
    print(json.dumps(updated_config, indent=2, ensure_ascii=False))

adapter_config.json 수정 중...

수정 전: base_model_name_or_path = TinyLlama/TinyLlama-1.1B-intermediate-step-1431k-3T
수정 후: base_model_name_or_path = TinyLlama/TinyLlama-1.1B-Chat-v1.0

✅ 수정 완료!

수정 결과 확인:
{
  "alpha_pattern": {},
  "auto_mapping": null,
  "base_model_name_or_path": "TinyLlama/TinyLlama-1.1B-Chat-v1.0",
  "bias": "none",
  "corda_config": null,
  "eva_config": null,
  "exclude_modules": null,
  "fan_in_fan_out": false,
  "inference_mode": true,
  "init_lora_weights": true,
  "layer_replication": null,
  "layers_pattern": null,
  "layers_to_transform": null,
  "loftq_config": {},
  "lora_alpha": 128,
  "lora_bias": false,
  "lora_dropout": 0.1,
  "megatron_config": null,
  "megatron_core": "megatron.core",
  "modules_to_save": null,
  "peft_type": "LORA",
  "qalora_group_size": 16,
  "r": 64,
  "rank_pattern": {},
  "revision": null,
  "target_modules": [
    "o_proj",
    "up_proj",
    "q_proj",
    "v_proj",
    "gate_proj",
    "down_proj",
    "k_proj"
  ],
  "target_pa

### **추론**
- 앞에서 정의한 채팅 프롬프트 템플릿을 사용해 모델 실행하기

In [14]:
from transformers import pipeline

# 사전에 정의된 프롬프트 템플릿을 사용합니다.
prompt = """<|user|>
Tell me something about Large Language Models.</s>
<|assistant|>
"""

# 인스트럭션 튜닝된 모델을 실행합니다.
pipe = pipeline(task="text-generation", model=merged_model, tokenizer=tokenizer)
print(pipe(prompt)[0]["generated_text"])

Device set to use cuda:0


<|user|>
Tell me something about Large Language Models.</s>
<|assistant|>
Large Language Models are machine learning models which can understand and generate text based on large datasets. They are trained on large amounts of text data and are often used for text generation tasks such as chatbots, machine translation, and NLP.

Large Language Models can generate text from a variety of sources, including text from books, tweets, social media posts, and webpages. They have been shown to be effective in a variety of natural language processing use cases, including text summarization, question answering, and dialogue generation.

Some of the notable Large Language Models include BART, GPT-3, and GPT-neo. BART is a Bidirectional Transformer Model which uses a combination of RNN and LSTM layers to generate text. GPT-3 is a generative pre-trained language model which can generate text in a variety of styles and genres, including text summarization, chatbots, and dialogue generation. GPT-neo is

### **[실습] 학습된 채팅 모델에 한글 질문 입력하기**

In [15]:
from transformers import pipeline

# 사전에 정의된 프롬프트 템플릿을 사용합니다.
prompt = """<|user|>
LLM에 대해 설명해줘.</s>
<|assistant|>
"""

# 인스트럭션 튜닝된 모델을 실행합니다.
pipe = pipeline(task="text-generation", model=merged_model, tokenizer=tokenizer)
print(pipe(prompt)[0]["generated_text"])

Device set to use cuda:0


<|user|>
LLM에 대해 설명해줘.</s>
<|assistant|>
LLM or Master of Laws (LLM) is a postgraduate degree that is often awarded to students who have completed a graduate degree in a law-related field. The LLM degree is a graduate program that provides students with the knowledge and skills necessary to practice law in a specific area of law. LLM programs focus on specialized fields of law, such as corporate and commercial law, intellectual property law, international law, and tax law.

LLM programs usually last between two and three years, and they often involve a combination of coursework and fieldwork or internships. The coursework in LLM programs typically covers a broad range of subjects, including legal theory, legal research and writing, legal ethics, and international law. Students may also be required to take relevant specialized courses, such as a course in the area of their interest or a course in a specific legal field.

LLM programs typically offer a wide range of specializations in la

### **[실습] 반복문 사용하여 챗봇 형태로 변경하기**

- 더 “대화스럽게” 만들고 싶다면 do_sample=True, temperature=0.7, top_p=0.9 같은 샘플링 설정을 시도



---



## **생성 모델 평가**

- 생성 모델을 평가하는 것은 매우 어렵다.
- 매우 다양한 사용 사례에서 생성 모델이 사용되기 때문에 하나의 지표로 판단하는 것은 위험하다.
- 확률적인 속성 때문에 생성 모델은 일관된 출력을 보장하지 않는다.
- 따라서 강력한 평가 방법이 필요하다.
- 참고 : https://gagadi.tistory.com/58

### **단어 수준 지표**

- 단어 수준(word-level) 평가 방법, 전통적인 기법
- 참조 데이터셋과 생성된 텍스트를 토큰 수준에서 비교함
- 대표적 단어 수준 지표
    - **혼잡도(perplexity)**, 1977
        - https://huggingface.co/spaces/evaluate-metric/perplexity
        - https://huggingface.co/docs/transformers/perplexity
        - 언어 모델이 텍스트를 얼마나 잘 예측하는지를 측정하는 지표
        - 입력 텍스트가 주어지면 모델은 다음에 올 토큰의 가능성을 예측하는데 혼잡오에서는 다음 토큰에 높은 확률을 부여할 때 모델이 잘 동작한다고 가정함
        - 일관성, 유창성, 창의성, 정확성을 고려하지 않는다.
    - ROUGE, 2004
        - https://huggingface.co/spaces/evaluate-metric/rouge
        - ROUGE(Recall-Oriented Understudy for Gisting Evaluation) Score
    - BLEU, 2002
        - https://huggingface.co/spaces/evaluate-metric/bleu
        - BLEU(BiLingual evaluation understudy) Score
    - BERTScore, 2019


### **벤치마크**

- 유명한 공개 벤치마크를 사용
- 이런 벤치마크는 기본적인 언어 이해뿐만 아니라 수학 문제 같이 복잡한 분석이 필요한 답변에 관한 정보를 제공함
- 벤치마크는 다양한 작업에서 모델이 얼마나 잘 수행되는지 이해하기 좋은 방법임
- 공개 벤치마크의 단점은 모델이 이런 벤치마크에서 최상의 답변을 내기 위해 과대적합될 수 있다는 것
- 일부 모델은 자연어 작업 이외에 프로그래밍 같은 영역에 특화되어 있음 --> 이런 모델은 HumanEval 같은 벤치마크로 평가되는 경향이 있음.
- 생성 모델을 위한 공개 벤치마크
    - **MMLU**
        - https://github.com/hendrycks/test
        - MMLU(Massive Multitask Language Understanding) 벤치마크는 분류, 질문 답변, 감성 분석을 포함해 57개의 작업에서 모델을 테스트함
    - **GLUE**
        - https://gluebenchmark.com/
        - GLUE(General Language Understanding Evaluation) 벤치마크는 다양한 난이도의 언어 이해 작업으로 구성됨
    - **TruthfulQA**
        - https://github.com/sylinrl/TruthfulQA
        - TruthfylQA는 모델이 생성한 텍스트의 진실성을 측정함
    - **GSM8k**
        - GSM8k 데이터셋은 초등학교 수준의 서술형 수학 문제를 담고 있음. 사람이 작성한 다양한 언어의 문제로 구성됨
    - **HellaSwag**
        - https://rowanzellers.com/hellaswag/
        - HellaSwag는 상식 추론을 평가하기 위한 도전적인 과제임. 모델이 답변하는 객관식 문제로 구성됨. 문제마다 네 개중 하나의 답을 선택할 수 있음
    - **HumanEval**
        - https://github.com/openai/human-eval
        - Chatbot Arena : https://lmarena.ai/
        - 164개의 프로그래밍 문제를 기반으로 생성된 코드를 평가하는데 사용됨

### **리더보드**

- 모델이 릴리스될 때마다 얼마나 잘 수행되는지 확인하기 위해 여러 벤치마크에서 평가할 것이다. 이를 위해 여러 벤치마크를 포함한 리더보드가 개발되었다.
- 대표적인 리더보드
    - **Open LLM Learderboard**:
        https://huggingface.co/spaces/open-llm-leaderboard/open_llm_leaderboard#/

### **자동 평가**
- LLM-as-judge :
    - Chatbot Arena : https://lmarena.ai/
    - 별도의 LLM이 다른 LLM의 생성된 텍스트의 품질 평가
    - 쌍별(Pairwise) 비교
    - 두 개의 다른 LLM이 질문에 대한 답을 생성하고 세 번째 LLM이 더느 답변이 더 좋은지 평가


### **사람 평가**

- 사람의 선호도 반영
- Chatbot Arena : https://lmarena.ai/



---



## **선호도 튜닝 (PPO/DPO)**

- 특정 답변을 더 선호하는 (사람의) 기호를 어떻게 LLM의 출력에 주입할 수 있을까?
    - 사람(선호도 평가자)에게 모델의 생성 품질을 평가하도록 요청할 수 있다.
    - 이 점수를 기반으로 모델을 업데이트하는 선호도 튜닝을 할 수 있다.



### 보상 모델 훈련
- 선호도 튜닝을 위한 세 단계
    1. 선호도 데이터 수집
    2. 보상 모델 훈련
    3. 보상 모델을 사용해 (선호도 평가자로 동작하는) LLM 미세 튜닝

- **PPO(Proximal Policy Optimization)**
    - 훈련된 보상 모델로 LLM을 미세 튜닝하는 방법
    - PPO는 지시 기반으로 튜닝된 LLM을 최적화하는데 널리 사용되는 강화학습 기법으로 기대하는 보상에서 LLM이 크게 빗나가지 않도록 한다.
    - 2022년 11월에 릴리스된 ChatGPT를 훈련하는데 사용

### 비보상 모델 훈련

- PPO의 단점은 보상 모델과 LLM을 둘 다 혼련해야하는 복잡한 방법이라는 점, 비용도 많이 든다.
- **DPO(Direct Preference Optimiztion)**
    - PPO의 대안으로 강화학습 방법을 사용하지 않는다.
    - 생성 품질을 평가하는데 보상 모델을 사용하는 대신 LLM이 스스로 수행하게 함
    - LLM의 복사본을 참조 모델로 사용해 승인된 생성과 거부된 생성의 품질에서 참조 모델과 훈련 가능한 모델 간의 차이를 판단함
    - DPO가 안정적임

## **DPO를 사용한 선호도 튜닝**

- **SFT + DPO**
    - 기본적인 채팅을 수행하도록 먼저 미세 튜닝하고
    - 그 다음에 사람의 선호도에 맞춰 답변하도록 정렬하는 방법
    - 두번의 훈련 루프 실행함 --> 두번에 걸쳐 파라미터 수정함

### **데이터 준비**

- 정렬 데이터에 템플릿 적용하기
- 사용 데이터셋
    - 각각의 프롬프트마다 승인된 생성과 거부된 생성을 담고 있는 데이터셋 사용
    - **argilla/distilabel-intel-orca-dpo-pairs**
    - https://huggingface.co/datasets/argilla/distilabel-intel-orca-dpo-pairs

In [16]:
from datasets import load_dataset

def format_prompt(example):
    """TinyLlama의 <|user|> 템플릿을 사용해 프롬프트를 구성합니다"""

    # 템플릿 포맷팅
    system = "<|system|>\n" + example['system'] + "</s>\n"
    prompt = "<|user|>\n" + example['input'] + "</s>\n<|assistant|>\n"
    chosen = example['chosen'] + "</s>\n"
    rejected = example['rejected'] + "</s>\n"

    return {
        "prompt": system + prompt,
        "chosen": chosen,
        "rejected": rejected,
    }

# 데이터셋에 템플릿을 적용하고 비교적 짧은 대답을 선택합니다
dpo_dataset = load_dataset("argilla/distilabel-intel-orca-dpo-pairs", split="train")
dpo_dataset = dpo_dataset.filter(
    lambda r:
        r["status"] != "tie" and
        r["chosen_score"] >= 8 and
        not r["in_gsm8k_train"]
)
dpo_dataset = dpo_dataset.map(format_prompt, remove_columns=dpo_dataset.column_names)
dpo_dataset

README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/79.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/12859 [00:00<?, ? examples/s]

Filter:   0%|          | 0/12859 [00:00<?, ? examples/s]

Map:   0%|          | 0/5922 [00:00<?, ? examples/s]

Dataset({
    features: ['chosen', 'rejected', 'prompt'],
    num_rows: 5922
})

### **모델 양자화**

- 베이스 모델을 로드하고 (앞에서 설정한 것과 동일한) QLoRA 설정
- 훈련에 필요한 VRAM 줄이기 위해 모델을 양자화함

In [17]:
from peft import AutoPeftModelForCausalLM
from transformers import BitsAndBytesConfig, AutoTokenizer

# 4-비트 양자화 설정 - QLoRA의 Q 단계
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,          # 4-비트 정밀도 모델 로드
    bnb_4bit_quant_type="nf4",  # 양자화 종류
    bnb_4bit_compute_dtype="float16",   # 계산 dtype
    bnb_4bit_use_double_quant=True,     # 이중 양자화 적용
)

# LoRA와 베이스 모델을 합칩니다.
model = AutoPeftModelForCausalLM.from_pretrained(
    "./TinyLlama-1.1B-qlora",
    low_cpu_mem_usage=True,
    device_map="auto",
    quantization_config=bnb_config,
)
# merged_model = model.merge_and_unload()

# LLaMA 토크나이저를 로드합니다.
model_name = "TinyLlama/TinyLlama-1.1B-intermediate-step-1431k-3T"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = "<PAD>"
tokenizer.padding_side = "left"

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

- 이전과 동일한 LoRA 설정으로 DPO 훈련 수행

In [18]:
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model

# LoRA 설정
peft_config = LoraConfig(
    lora_alpha=128,  # LoRA 스케일링
    lora_dropout=0.1,  # LoRA 층의 드롭아웃
    r=64,  # 랭크
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=  # 대상 층
     ['k_proj', 'gate_proj', 'v_proj', 'up_proj', 'q_proj', 'o_proj', 'down_proj']
)

# 훈련을 위해 모델을 준비합니다.
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)

/usr/local/lib/python3.12/dist-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/tuners/tuners_utils.py:196: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


### **훈련 설정**

- 훈련 매개변수 설정: SFTConfig-->DPOConfig
    - 추가 : warmup_ratio (초기 10%의 스텝 동안 학습률을 0 ~ learning_rate값까지 증가시킴
        -시작할 때(워밍업 기간 동안) 학습률을 작게 유지함으로써 학습률이 커지기 전에 모델이 데이터에 적응하여 유해한 발산으로 이어지지 않게 할 수 있다
    - 추가 : max_steps=200, --> 삭제: num_train_epochs=1

In [19]:
from trl import DPOConfig

output_dir = "./results"

# 훈련 매개변수
training_arguments = DPOConfig(
    output_dir=output_dir,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    optim="paged_adamw_32bit",
    learning_rate=1e-5,
    lr_scheduler_type="cosine",
    max_steps=200,
    logging_steps=10,
    fp16=True,
    gradient_checkpointing=True,
    warmup_ratio=0.1,
    beta=0.1,
    max_prompt_length=512,
    max_length=512
)

### **훈련**

In [20]:
from trl import DPOTrainer

# DPOTrainer 객체를 만듭니다.
dpo_trainer = DPOTrainer(
    model,
    args=training_arguments,
    train_dataset=dpo_dataset,
    processing_class=tokenizer,
    peft_config=peft_config
)

# DPO로 모델을 미세 튜닝합니다.
dpo_trainer.train()

# 어댑터를 저장합니다.
dpo_trainer.model.save_pretrained("TinyLlama-1.1B-dpo-qlora")

/usr/local/lib/python3.12/dist-packages/peft/tuners/lora/bnb.py:348: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/tuners/tuners_utils.py:196: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


Extracting prompt in train dataset:   0%|          | 0/5922 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/5922 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/5922 [00:00<?, ? examples/s]

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 0}.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.12/dist-packages/torch/utils/checkpoint.py:85: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


Step,Training Loss
10,0.691100
20,0.644100
30,0.549600
40,0.595000
50,0.541100
60,0.564800
70,0.507800
80,0.457300
90,0.432700
100,0.604200


In [21]:
from peft import PeftModel

# LoRA와 베이스 모델을 합칩니다.
model = AutoPeftModelForCausalLM.from_pretrained(
    "./TinyLlama-1.1B-qlora",
    low_cpu_mem_usage=True,
    device_map="auto",
)
sft_model = model.merge_and_unload()

# DPO LoRA와 SFT 모델을 합칩니다.
dpo_model = PeftModel.from_pretrained(
    sft_model,
    "./TinyLlama-1.1B-dpo-qlora",
    device_map="auto",
)
dpo_model = dpo_model.merge_and_unload()

/usr/local/lib/python3.12/dist-packages/peft/tuners/tuners_utils.py:196: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/peft_model.py:585: UserWarning: Found missing adapter keys while loading the checkpoint: ['base_model.model.model.layers.0.self_attn.q_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.q_proj.lora_B.default.weight', 'base_model.model.model.layers.0.self_attn.k_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.k_proj.lora_B.default.weight', 'base_model.model.model.layers.0.self_attn.v_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.v_proj.lora_B.default.weight', 'base_model.model.model.layers.0.self_attn.o_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.o_proj.lora_B.default.weight', 'base_model.model.mod

In [22]:
from transformers import pipeline

# 사전에 정의된 프롬프트 템플릿을 사용합니다.
prompt = """<|user|>
Tell me something about Large Language Models.</s>
<|assistant|>
"""

# 인스트럭션 튜닝된 모델을 실행합니다.
pipe = pipeline(task="text-generation", model=dpo_model, tokenizer=tokenizer)
print(pipe(prompt)[0]["generated_text"])

Device set to use cuda:0


<|user|>
Tell me something about Large Language Models.</s>
<|assistant|>
Large Language Models (LLMs) are artificial intelligence ( ai) models designed to learn and generate human-like language. They are used in various applications, including natural language processing ( nlp), machine translation ( mt), and chatbots. Here are some key features of LLMs:

1. Large Vocabulary: LLMs have a vast collection of words and phrases that can be used to generate realistic and accurate natural language. They can learn from large amounts of text and data, including Wikipedia, news articles, and social media.

2. Deep Learning: LLMs are based on deep learning algorithms, which allow them to learn from large datasets and improve their performance over time. They can learn from both text and context, making them more versatile and effective in generating natural-sounding language.

3. Transfer Learning: LLMs can also learn from each other, transferring their knowledge and skills to improve their per

### **[실습] 학습된 채팅 모델에 한글 질문 입력하기**

In [22]:
from transformers import pipeline

# 사전에 정의된 프롬프트 템플릿을 사용합니다.
prompt = """<|user|>
거대 규모 LLM에 대해 설명해줘.</s>
<|assistant|>
"""

# 인스트럭션 튜닝된 모델을 실행합니다.
pipe = pipeline(task="text-generation", model=dpo_model, tokenizer=tokenizer)
print(pipe(prompt)[0]["generated_text"])

- **요약**
    - LoRA 기법을 통해 PEFT를 사용해 미세 튜징 수행
    - 모델과 어댑터의 파라미터에 필요한 메모리 제약을 줄이는 기법인 QLoRA 설명
    - SFT + DPO
        - 기본적인 채팅을 수행하도록 먼저 미세 튜닝하고
        - 그 다음에 사람의 선호도에 맞춰 답변하도록 정렬하는 방법
        - 두번의 훈련 루프 실행함 --> 두번에 걸쳐 파라미터 수정함
    - 최근
        - SFT + DPO지만 한 번의 훈련 과정로 결함한 ORPO(Odds Ration Preference Optimization)가 사용됨